In [1]:
# start tensorflow interactiveSession                           F1 = 0.7903857746
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 7500])
y_ = tf.placeholder("float", [None, 3])


# first convolutinal layer
w_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 50, 50, 3])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_RGB/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.005
for i in range(64000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train accuracy 0.24
step 200, train accuracy 0.6
step 400, train accuracy 0.65
step 600, train accuracy 0.65
step 800, train accuracy 0.68
step 1000, train accuracy 0.61
step 1200, train accuracy 0.68
step 1400, train accuracy 0.63
step 1600, train accuracy 0.74
step 1800, train accuracy 0.74
step 2000, train accuracy 0.61
step 2200, train accuracy 0.68
step 2400, train accuracy 0.64
step 2600, train accuracy 0.68
step 2800, train accuracy 0.59
step 3000, train accuracy 0.74
step 3200, train accuracy 0.75
step 3400, train accuracy 0.78
step 3600, train accuracy 0.64
step 3800, train accuracy 0.84
step 4000, train accuracy 0.68
step 4200, train accuracy 0.62
step 4400, train accuracy 0.74
step 4600, train accuracy 0.75
step 4800, train accuracy 0.68
step 5000, train accuracy 0.72
step 5200, train accuracy 0.66
step 5400, train accuracy 0.72
step 5600, train accuracy 0.76
step 5800, train accuracy 0.66
step

step 50000, train accuracy 0.97
step 50200, train accuracy 0.97
step 50400, train accuracy 0.98
step 50600, train accuracy 0.99
step 50800, train accuracy 0.99
step 51000, train accuracy 0.99
step 51200, train accuracy 0.99
step 51400, train accuracy 0.98
step 51600, train accuracy 0.95
step 51800, train accuracy 0.97
step 52000, train accuracy 0.99
step 52200, train accuracy 0.98
step 52400, train accuracy 0.98
step 52600, train accuracy 0.99
step 52800, train accuracy 0.99
step 53000, train accuracy 1
step 53200, train accuracy 1
step 53400, train accuracy 0.99
step 53600, train accuracy 1
step 53800, train accuracy 0.98
step 54000, train accuracy 1
step 54200, train accuracy 0.99
step 54400, train accuracy 0.99
step 54600, train accuracy 0.97
step 54800, train accuracy 0.99
step 55000, train accuracy 0.99
step 55200, train accuracy 0.98
step 55400, train accuracy 1
step 55600, train accuracy 0.99
step 55800, train accuracy 0.99
step 56000, train accuracy 1
step 56200, train accuracy

In [2]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_RGB/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:1.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_RGB_pix_pred_64k_batch_new.csv', index = False)

In [1]:
#使用Adam优化器更新参数                                          F1 = 0.8126883665
# start tensorflow interactiveSession                           
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 7500])
y_ = tf.placeholder("float", [None, 3])


# first convolutinal layer
w_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 50, 50, 3])

h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer
w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_RGB/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.005
for i in range(10000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train accuracy 0.55
step 200, train accuracy 0.71
step 400, train accuracy 0.74
step 600, train accuracy 0.75
step 800, train accuracy 0.84
step 1000, train accuracy 0.72
step 1200, train accuracy 0.76
step 1400, train accuracy 0.79
step 1600, train accuracy 0.88
step 1800, train accuracy 0.81
step 2000, train accuracy 0.78
step 2200, train accuracy 0.87
step 2400, train accuracy 0.86
step 2600, train accuracy 0.91
step 2800, train accuracy 0.87
step 3000, train accuracy 0.97
step 3200, train accuracy 0.94
step 3400, train accuracy 0.95
step 3600, train accuracy 0.93
step 3800, train accuracy 0.96
step 4000, train accuracy 0.94
step 4200, train accuracy 0.95
step 4400, train accuracy 0.96
step 4600, train accuracy 0.98
step 4800, train accuracy 0.94
step 5000, train accuracy 0.99
step 5200, train accuracy 0.98
step 5400, train accuracy 0.97
step 5600, train accuracy 0.98
step 5800, train accuracy 1
step 6

In [2]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_RGB/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:1.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_RGB_pix_pred_10k_batch_adam.csv', index = False)

##### 用2个3 * 3卷积层替代一个5 * 5卷积层

In [1]:
#使用Adam优化器更新参数                                         F1 = 0.8169077758 
# start tensorflow interactiveSession                           
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 7500])
y_ = tf.placeholder("float", [None, 3])
x_image = tf.reshape(x, [-1, 50, 50, 3])
keep_prob = tf.placeholder("float")

# first convolutinal layer
w_conv1_1 = weight_variable([3, 3, 3, 32])
b_conv1_1 = bias_variable([32])

w_conv1_2 = weight_variable([3, 3, 32, 32])
b_conv1_2 = bias_variable([32])

h_conv1_1 = tf.nn.relu(conv2d(x_image, w_conv1_1) + b_conv1_1)
h_conv1_2 = tf.nn.relu(conv2d(h_conv1_1, w_conv1_2) + b_conv1_2)
h_pool1 = max_pool_2x2(h_conv1_2)

# second convolutional layer
w_conv2_1 = weight_variable([3, 3, 32, 64])
b_conv2_1 = bias_variable([64])

w_conv2_2 = weight_variable([3, 3, 64, 64])
b_conv2_2 = bias_variable([64])

h_conv2_1 = tf.nn.relu(conv2d(h_pool1, w_conv2_1) + b_conv2_1)
h_conv2_2 = tf.nn.relu(conv2d(h_conv2_1, w_conv2_2) + b_conv2_2)
h_pool2 = max_pool_2x2(h_conv2_2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_RGB/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.005
for i in range(10000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, train accuracy 0.34
step 200, train accuracy 0.68
step 400, train accuracy 0.73
step 600, train accuracy 0.76
step 800, train accuracy 0.76
step 1000, train accuracy 0.7
step 1200, train accuracy 0.77
step 1400, train accuracy 0.74
step 1600, train accuracy 0.85
step 1800, train accuracy 0.81
step 2000, train accuracy 0.75
step 2200, train accuracy 0.85
step 2400, train accuracy 0.82
step 2600, train accuracy 0.86
step 2800, train accuracy 0.79
step 3000, train accuracy 0.92
step 3200, train accuracy 0.94
step 3400, train accuracy 0.9
step 3600, train accuracy 0.91
step 3800, train accuracy 0.92
step 4000, train accuracy 0.96
step 4200, train accuracy 0.91
step 4400, train accuracy 0.93
step 4600, train accuracy 0.97
step 4800, train accuracy 0.93
step 5000, train accuracy 0.98
step 5200, train accuracy 0.96
step 5400, train accuracy 0.96
step 5600, train accuracy 0.99
step 5800, train accuracy 0.99
step 

KeyboardInterrupt: 

In [2]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_RGB/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:1.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_RGB_pix_pred_10k_batch_adam_3_3conv.csv', index = False)

##### 增加两层卷积层，增大batch_size

In [1]:
#使用Adam优化器更新参数                                         F1 = 0.8196202532
# start tensorflow interactiveSession                           
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 7500])
y_ = tf.placeholder("float", [None, 3])
x_image = tf.reshape(x, [-1, 50, 50, 3])
keep_prob = tf.placeholder("float")

# first convolutinal layer
w_conv1_1 = weight_variable([3, 3, 3, 32])
b_conv1_1 = bias_variable([32])

w_conv1_2 = weight_variable([3, 3, 32, 32])
b_conv1_2 = bias_variable([32])

h_conv1_1 = tf.nn.relu(conv2d(x_image, w_conv1_1) + b_conv1_1)
h_conv1_2 = tf.nn.relu(conv2d(h_conv1_1, w_conv1_2) + b_conv1_2)
h_pool1 = max_pool_2x2(h_conv1_2)

# second convolutional layer
w_conv2_1 = weight_variable([3, 3, 32, 64])
b_conv2_1 = bias_variable([64])

w_conv2_2 = weight_variable([3, 3, 64, 64])
b_conv2_2 = bias_variable([64])

h_conv2_1 = tf.nn.relu(conv2d(h_pool1, w_conv2_1) + b_conv2_1)
h_conv2_2 = tf.nn.relu(conv2d(h_conv2_1, w_conv2_2) + b_conv2_2)
h_pool2 = max_pool_2x2(h_conv2_2)

# third conv layer
w_conv3_1 = weight_variable([3, 3, 64, 128])
b_conv3_1 = bias_variable([128])

w_conv3_2 = weight_variable([3, 3, 128, 128])
b_conv3_2 = bias_variable([128])

h_conv3_1 = tf.nn.relu(conv2d(h_pool2, w_conv3_1) + b_conv3_1)
h_conv3_2 = tf.nn.relu(conv2d(h_conv3_1, w_conv3_2) + b_conv3_2)
h_pool3 = max_pool_2x2(h_conv3_2)



h_pool2_flat = tf.reshape(h_pool3, [-1, 7*7*128])
# densely connected layer
w_fc1 = weight_variable([7*7*128, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_RGB/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.02
for i in range(10000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})


step 0, train accuracy 0.103015
step 200, train accuracy 0.683417
step 400, train accuracy 0.728643
step 600, train accuracy 0.763819
step 800, train accuracy 0.801508
step 1000, train accuracy 0.809045
step 1200, train accuracy 0.821608
step 1400, train accuracy 0.871859
step 1600, train accuracy 0.909548
step 1800, train accuracy 0.91206
step 2000, train accuracy 0.922111
step 2200, train accuracy 0.937186
step 2400, train accuracy 0.964824
step 2600, train accuracy 0.9799
step 2800, train accuracy 0.9799
step 3000, train accuracy 0.987437
step 3200, train accuracy 0.987437
step 3400, train accuracy 0.997487
step 3600, train accuracy 0.997487
step 3800, train accuracy 0.997487
step 4000, train accuracy 0.997487
step 4200, train accuracy 1
step 4400, train accuracy 1
step 4600, train accuracy 1


KeyboardInterrupt: 

In [3]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_RGB/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:1.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_RGB_pix_pred_10k_batch_adam_3_3_3conv.csv', index = False)

##### 保持4+2CNN结构，增大batch_size

In [2]:
#使用Adam优化器更新参数                                         F1 = 0.81285
# start tensorflow interactiveSession                           
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image

sess = tf.InteractiveSession()

#####################################################     Net Define     ##################################################### 

# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Create the model
# placeholder
x = tf.placeholder("float", [None, 7500])
y_ = tf.placeholder("float", [None, 3])
x_image = tf.reshape(x, [-1, 50, 50, 3])
keep_prob = tf.placeholder("float")

# first convolutinal layer
w_conv1_1 = weight_variable([3, 3, 3, 32])
b_conv1_1 = bias_variable([32])

w_conv1_2 = weight_variable([3, 3, 32, 32])
b_conv1_2 = bias_variable([32])

h_conv1_1 = tf.nn.relu(conv2d(x_image, w_conv1_1) + b_conv1_1)
h_conv1_2 = tf.nn.relu(conv2d(h_conv1_1, w_conv1_2) + b_conv1_2)
h_pool1 = max_pool_2x2(h_conv1_2)

# second convolutional layer
w_conv2_1 = weight_variable([3, 3, 32, 64])
b_conv2_1 = bias_variable([64])

w_conv2_2 = weight_variable([3, 3, 64, 64])
b_conv2_2 = bias_variable([64])

h_conv2_1 = tf.nn.relu(conv2d(h_pool1, w_conv2_1) + b_conv2_1)
h_conv2_2 = tf.nn.relu(conv2d(h_conv2_1, w_conv2_2) + b_conv2_2)
h_pool2 = max_pool_2x2(h_conv2_2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 13*13*64])
# densely connected layer
w_fc1 = weight_variable([13*13*64, 1024])
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

# dropout
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout layer
w_fc2 = weight_variable([1024, 3])
b_fc2 = bias_variable([3])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

# train and evaluate the model
#交叉熵作为损失函数
delta = 1e-7
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv+delta))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())


#####################################################       Train     ##################################################### 

image_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train_RGB/'
data_path = 'E:/Age Detection of Indian Actors/train_DETg9GD/Train.csv'

data = pd.read_csv(data_path)
data = pd.get_dummies(data,columns = ['Class'])


def get_batch(data,batch_rate,seed):
    """
    本函数实现从训练集中随机抽取部分图像，格式适合输入tensorflow，同时得到样本label的独热编码
    input:
        batch_rate:    batch占训练集数据的比例
        data:          DataFrame       train.csv
        seed:          随机种子
    output:
        train_x:        训练图像tensor
        train_y:        训练图像one_hot编码的label
    """
    sample = data.sample(frac = batch_rate, random_state = seed)
    sample.index = list(range(0,sample.shape[0]))
    one_hot_sample = sample.drop(['ID'], axis = 1)
    train_y = one_hot_sample.values
    train_x = []
    for i in range(0,sample.shape[0]):
        image = image_path + sample.ID[i]
        im = np.array(Image.open(image))/255.0
        im = im.flatten()
        train_x.append(im)
    train_x = np.array(train_x)
    
    return train_x,train_y
    
    
    
    
    
batch_rate = 0.02
for i in range(5000):
    batch_x,batch_y = get_batch(data,batch_rate,i)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob:1.0})
        print ("step %d, train accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch_x, y_:batch_y, keep_prob:0.5})


E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, train accuracy 0.236181
step 200, train accuracy 0.673367
step 400, train accuracy 0.703518
step 600, train accuracy 0.78392
step 800, train accuracy 0.80402
step 1000, train accuracy 0.809045
step 1200, train accuracy 0.81407
step 1400, train accuracy 0.864322
step 1600, train accuracy 0.894472
step 1800, train accuracy 0.909548
step 2000, train accuracy 0.929648
step 2200, train accuracy 0.934673
step 2400, train accuracy 0.944724
step 2600, train accuracy 0.969849
step 2800, train accuracy 0.984925
step 3000, train accuracy 0.98995
step 3200, train accuracy 0.987437
step 3400, train accuracy 0.992462
step 3600, train accuracy 0.997487
step 3800, train accuracy 0.98995
step 4000, train accuracy 0.994975
step 4200, train accuracy 1
step 4400, train accuracy 1
step 4600, train accuracy 1


KeyboardInterrupt: 

In [3]:
image_path = 'E:/Age Detection of Indian Actors/test_Bh8pGW3/Test_RGB/'

test = pd.read_csv('E:/Age Detection of Indian Actors/test_Bh8pGW3/test.csv')

test_x = []
for i in range(0,test.shape[0]):
    image = image_path + test.ID[i]
    im = np.array(Image.open(image))/255.0
    im = im.flatten()
    test_x.append(im)
test_x = np.array(test_x)
test_y = np.zeros((test.shape[0],3))

pred = tf.argmax(y_conv, 1)

test_x_0 = test_x[0:1500]
test_y_0 = test_y[0:1500]
P_0 = pred.eval(feed_dict={x:test_x_0, y_:test_y_0, keep_prob:1.0})

test_x_1 = test_x[1500:3000]
test_y_1 = test_y[1500:3000]
P_1 = pred.eval(feed_dict={x:test_x_1, y_:test_y_1, keep_prob:1.0})

test_x_2 = test_x[3000:4500]
test_y_2 = test_y[3000:4500]
P_2 = pred.eval(feed_dict={x:test_x_2, y_:test_y_2, keep_prob:1.0})

test_x_3 = test_x[4500:]
test_y_3 = test_y[4500:]
P_3 = pred.eval(feed_dict={x:test_x_3, y_:test_y_3, keep_prob:1.0})

P = np.hstack([P_0,P_1,P_2,P_3])

test['pred'] = P

Dict = {0:'MIDDLE',1:'OLD', 2:'YOUNG'}
Class = []
for i in range(0,test.shape[0]):
    Class.append(Dict.get(test['pred'][i]))
    
pred = pd.DataFrame({'Class':Class,'ID':test.ID})
pred.head()

pred.to_csv('50_RGB_pix_pred_10k_batch_adam_3_3_2conv.csv', index = False)